In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from sherpa_ai.memory import Belief
from react.states import get_actions, add_state_machine
from utils import load_processed_dataset
import random
from langchain_openai import ChatOpenAI
from sherpa_ai.agents.qa_agent import QAAgent
from sherpa_ai.events import Event, EventType
from clevr_policy import ReactPolicy
from tqdm.notebook import tqdm
import pandas as pd
from datasets import load_dataset

In [4]:
dataset = load_dataset("Dogdays/clevr_subset", token=True)["train"]

In [5]:
model_name = "gpt-4o-mini"
llm = ChatOpenAI(model=model_name)

Next: Run the react state machine with Clevr example

In [48]:
add_state_machine(belief, action_map)

---
State Machine
---
stateDiagram-v2
  direction LR
  classDef s_default fill:white,color:black
  classDef s_inactive fill:white,color:black
  classDef s_parallel color:black,fill:white
  classDef s_active color:red,fill:darksalmon
  classDef s_previous color:blue,fill:azure
  
  state "Start" as Start
  Class Start s_active
  state "Reasoning" as Reasoning
  Class Reasoning s_default
  state "Finish" as Finish
  Class Finish s_default
  
  Start --> Reasoning: start
  Reasoning --> Reasoning: filter_with_attribute | query_attribute | get_related_objects | get_same_objects | count_all_objects
  Reasoning --> Finish: answer
  [*] --> Start


In [43]:
results = []

AGENT_DESCRIPTION = """
You are a question answering assistant helping users to find answers to their questions based on a specific scene.
Each object in the scene contain the following properties: color, size, shape, material, and a unique identifier.
The properties are from a fixed set of values:
– Size: One of large or small.
– Color: One of gray, red, blue, green, brown, purple, cyan, or yellow.
– Shape: One of cube (block), sphere, or cylinder.
– Material: One of rubber (matte) or metal (shinning).
- Unique identifier: The index of the object in the scene, starting from 0.

Scene: {scene}

Objects in the scene also have the following relationships: left, right, front or behind.

Use the available actions to interact with the scene and collect information to answer the question.
When provide action arguments, ONLY use the values from the fixed set of values above.
"""

use_scene = True

for sample in tqdm(dataset.select(range(100))):
    scene = sample["scene"]
    test_question = sample["question"]

    belief = Belief()
    action_map = get_actions(belief, llm)
    add_state_machine(belief, action_map)

    belief.set("scene", scene)
    if use_scene:
        belief.set("agent_scene", scene)

    agent_description = AGENT_DESCRIPTION.format(scene=scene)

    policy = ReactPolicy(
        role_description=agent_description,
        output_instruction="Output the answer to the question.",
        llm=llm,
    )

    agent = QAAgent(
        llm=llm,
        belief=belief,
        description=agent_description,
        num_runs=10,
        policy=policy,
    )

    belief.set_current_task(
        Event(
            EventType.task, "user", f"Answer the question: {test_question}."
        )
    )
    agent.run()

    if belief.state_machine.state != "Finish":
        belief.state_machine.answer()

    result = [str(event) for event in belief.internal_events]
    answer = belief.get("answer_action", "No answer found.")
    results.append((result, answer))

  0%|          | 0/100 [00:00<?, ?it/s]

2024-11-26 23:31:27.435 | ERROR    | sherpa_ai.agents.base:act:351 - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Traceback (most recent call last):

  File "c:\Users\chenp\anaconda3\envs\sherpa\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x000001F35A907050, file "c:\Users\chenp\anaconda3\envs\sherpa\lib\site-packages\ipykernel_launcher....
           └ <function _run_code at 0x000001F35A90B2E0>

  File "c:\Users\chenp\anaconda3\envs\sherpa\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
         └ <code object <module> at 0x000001F35A907050, 

In [29]:
real_answers = [q["answer"] for q in dataset]

In [42]:
results[1]

(["filter_with_attribute: EventType.action - Action: filter_with_attribute starts, Args: {'attributes': {'color': 'green', 'shape': 'cube'}}",
  'filter_with_attribute: EventType.action_output - Action: filter_with_attribute finishes, Observation: []',
  'answer: EventType.action - Action: answer starts, Args: {}',
  "answer: EventType.action_output - Action: answer finishes, Observation: {'answer': 0}"],
 {'answer': 0})

In [44]:
predicted_answers = []

for result in results:
    if type(result[1]) == str:
        predicted_answers.append(result[1])
    else:
        predicted_answers.append(result[1]["answer"])

In [45]:
correct = [1 if str(p) == str(a) else 0 for p, a in zip(predicted_answers, real_answers)]
print(len(correct), sum(correct), sum(correct) / len(correct))

100 85 0.85


In [46]:
df = pd.DataFrame({
    "predicted": [r[1] for r in results],
    "reasoning": [r[0] for r in results],
    "actual": real_answers
})

df.to_csv(f"react_results_{model_name}.csv", index=False)

In [6]:
results_dict = []
for log, question in results:
    results_dict.append({
        "log": log,
        "question": question
    })

df = pd.DataFrame(results_dict)
df.to_csv("results_react_gpt-4o.csv", index=False)

['count_all_objects: EventType.action - Action: count_all_objects starts, Args: {}',
 'count_all_objects: EventType.action_output - Action: count_all_objects finishes, Observation: 10',
 "filter_with_attribute: EventType.action - Action: filter_with_attribute starts, Args: {'object_ids': [], 'attribute': 'material', 'value': 'rubber'}",
 'filter_with_attribute: EventType.action_output - Action: filter_with_attribute finishes, Observation: [3, 5, 6, 7, 9]',
 "filter_with_attribute: EventType.action - Action: filter_with_attribute starts, Args: {'object_ids': [], 'attribute': 'material', 'value': 'metal'}",
 'filter_with_attribute: EventType.action_output - Action: filter_with_attribute finishes, Observation: [0, 1, 2, 4, 8]',
 "answer: EventType.action - Action: answer starts, Args: {'answer': 'No, there are more matte objects than shiny objects.'}",
 'answer: EventType.action_output - Action: answer finishes, Observation: No, there are more matte objects than shiny objects.']

In [ ]:
print(test_question['question'])

how many cylinders are there that are in front of a green cube?


2024-10-15 23:41:24.088 | WARNING  | sherpa_ai.agents.base:run:96 - ```start
 Input: {}...```
c:\Users\chenp\anaconda3\envs\sherpa\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
2024-10-15 23:41:26.803 | WARNING  | sherpa_ai.agents.base:run:96 - ```filter_with_attribute
 Input: {'object_ids': [], 'attribute': 'shape', 'value': 'cylinder'}...```
2024-10-15 23:41:26.804 | WARNING  | sherpa_ai.actions.base:__call__:150 - Action output: [0, 5, 6, 8, 9]
2024-10-15 23:41:29.358 | WARNING  | sherpa_ai.agents.base:run:96 - ```get_related_objects
 Input: {'object_id': 8, 'relation': 'front'}...```
2024-10-15 23:41:29.359 | WARNING  | sherpa_ai.actions.base:__call__:150 - Action output: [0, 1, 2, 3, 5, 6, 7, 9]
2024-10-15 23:41:34.642 | WARNING  | sherpa_ai.agents.base:run:96 - ```filter_with_attribute
 Input: {'objec

PolicyOutput(action={
    "name": "answer",
    "args": {
        "answer": "Answer to the question, type: str"
    },
    "usage": "Output answer to the question Transit the state from Reasoning to Finish"
}, args={'answer': '2'})

In [ ]:
test_question

{'split': 'val',
 'image_filename': 'CLEVR_val_003661.png',
 'answer': '0',
 'question': 'how many cylinders are there that are in front of a green cube?',
 'image_index': 3661}